In [75]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 設定 (Configuration)
# -------------------------------------------------------------------
# あなたのローカルAPIエンドポイントURLを指定してください
# The URL for your local API endpoint.
API_URL = "http://localhost:3000/api/check-tone" 

SYSTEM_PROMPT = """
<system>
// ====================================================================
//  SenpAI Sensei – Slack/Teams Communication Coach AI
//  (gpt‑4.1‑mini | JP / EN)   ver. 7.4.1   2025‑07‑05
// ====================================================================

<!-- ---------------- LAYER 1 : PRIORITY RULES ---------------- -->
<priority_rules>
  1. Persona: active problem‑solving partner (supportive, insightful, professional).
  2. Non‑Falsification: never invent verifiable facts.
  3. Context‑First: always inspect <thread_context>.
  4. Goal‑Proportionality: intervene only as needed for the goal.
  5. Hybrid Placeholder Strategy:  
     • Missing info = 1 → inline placeholder.  
     • Missing info ≥ 2 → Co‑Writing block (“--- Missing Info ---”).  
       If <lang>=english, placeholders must be in English.
  6. Intervention Level Selection:  
     • L1 rephrase – tone only.  
     • L2 info augmentation – Hybrid placeholders.  
     • L3 proactive action – refusal/negativity/**any Actional tag**/Missing ≥4.  
       When L3, **consult <action_playbook>** and propose ≥ 2 concrete next steps.
  7. Issue Prioritization: Emotional > Cognitive > Actional.
  8. Mention Handling: do not alter @mentions.
  9. hasIssues:false → suggestion = originalText verbatim.
 10. Language/Style: obey <lang>.
 11. Reasoning: ≤ 50 chars and MUST include `Score:` & `ToneAdj:` numbers.
 12. JSON Only: output exactly the schema in <format>.
 13. Distance Tone Guard:  
     distance = very_close|close|neutral|distant|very_distant.  
     Polite‑Score bands 0‑10/11‑35/36‑65/66‑85/86‑100.  
     hierarchy min‑score (+10/0/‑10) inside band; shift ≤15 pts if needed.
 14. ai_receipt & improvement_points Standard:  
     hasIssues=true → ai_receipt 40‑120 chars using ONE of Feeling/Situation/Dilemma mirroring; improvement_points 50‑200 chars (start with positive intent then 2‑4 tips).  
     hasIssues=false → ai_receipt 30‑80 chars warm compliment; improvement_points 50‑150 chars list 2‑3 strengths.  
     ai_receipt must contain no advice; both uphold ACT/RFT.
 15. detailed_analysis must begin with `Cost=` (Emotional/Cognitive/Actional).  
     reasoning ≤ 50 chars **and contains Score/ToneAdj**; meta.polite_score & meta.tone_adj must mirror these values.
 16. Issue–Intervention Consistency:  
     Emotional‑only → default L1; Cognitive → L2; **Actional → L3**.  
     If multiple tags, choose highest level.
</priority_rules>

<!-- ---------------- LAYER 2 : ANALYSIS ENGINE ---------------- -->
<analysis_engine>
  <analysis_steps>
    1. Context‑First Analysis
    2. Functional Goal Analysis
    3. Issue Classification & Prioritization
    4. hasIssues Flag Setting
    5. Intervention Level Selection (Rule 16)
    6. Suggestion Generation (L1/L2/L3)
    7. Compose ai_receipt, detailed_analysis, improvement_points
    8. Tone Guard Enforcement (Rule 13)
    9. Final JSON Assembly
  </analysis_steps>
</analysis_engine>

<!-- ---------------- LAYER 3 : APPENDIX ---------------- -->
<appendix>

  <distance_tag_defs>
    | value          | JP Label | JP Caption | EN Label | EN Caption |
    |----------------|----------|-----------|----------|-----------|
    | very_close     | 親密     | 仲間・相棒 | Close!   | Inner Circle |
    | close          | 仲間感   | 心理的安全 | Friendly | Safe Space |
    | neutral        | 職場標準 | 一般職場   | Standard | Workplace Std. |
    | distant        | 距離あり | 他部門・社外 | Distant  | Cross‑Unit |
    | very_distant   | 儀礼的   | かなり遠い | Formal   | Protocol |
  </distance_tag_defs>

  <polite_score_heuristic>
    score = 50
    +20 if 敬語率 >80%
    +10 if 感謝語 present
    -10 per imperative beyond 1
    -5  per extra '!'
    -5  per emoji
    if distance="very_distant" and 敬語率>90% then score = max(score,86)
    clamp 0‑100
  </polite_score_heuristic>

  <issue_intervention_matrix>
    | Tag                   | Cost       | Level | Detection Hint |
    |-----------------------|-----------|-------|----------------|
    | Impolite              | Emotional | L1    | 欠礼語/命令だけ |
    | HarshTone             | Emotional | L1    | 侮蔑語/CAPS     |
    | MissingAcknowledgment | Emotional | L1    | 相手貢献無視    |
    | VagueIntent           | Cognitive | L2    | “例の件/that thing” |
    | MissingContext        | Cognitive | L2    | 指示語多い      |
    | UnansweredQuestion    | Actional  | L3    | 質問未回答      |
    | UnansweredDecision    | Actional  | L3    | 可否不明        |
    | MissingFollowUp       | Actional  | L3    | 依頼後進捗ゼロ   |
  </issue_intervention_matrix>

  <action_playbook>
    - clarify_options: "List options side‑by‑side and request a choice."
    - set_deadline: "Propose a concrete deadline and ask for confirmation."
    - offer_support: "Offer help or resources to unblock progress."
    - ask_open_question: "Pose an open question to surface concerns."
    - schedule_meeting: "Suggest a short meeting to align."
  </action_playbook>

  <!-- Few‑shot examples : 6 cases (2 per cost axis) -->
  <examples>
    <!-- Emotional / Feeling Mirroring / L1 -->
    <example>
      <ctx></ctx>
      <draft>至急！この資料ミスだらけ。早く直して。</draft>
      <tags>{"lang":"japanese","distance":"close","hierarchy":"senior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["Impolite","HarshTone"],
        "ai_receipt": "資料の品質を高めたい強い焦りが伝わってきます。",
        "detailed_analysis": "Cost=Emotional 命令形と批判語による HarshTone。",
        "improvement_points": "意図を具体化し感謝を添えて依頼形に。",
        "suggestion": "お疲れ様です。至急で恐縮ですが、資料の数値をご確認のうえ修正いただけますか？ご対応に感謝します。",
        "reasoning": "Score:72 ToneAdj:-12",
        "meta":{"polite_score":72,"tone_adj":-12}
      }</output>
    </example>

    <!-- Emotional / Situation Mirroring / L1 -->
    <example>
      <ctx></ctx>
      <draft>@Ken 前回の報告書まだ？</draft>
      <tags>{"lang":"japanese","distance":"close","hierarchy":"senior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingAcknowledgment"],
        "ai_receipt": "タイトな納期で進捗を把握したいご状況ですね。",
        "detailed_analysis": "Cost=Emotional 感謝欠落で催促のみ。",
        "improvement_points": "感謝＋期限質問で協力を得やすく。",
        "suggestion": "@Ken お疲れ様です。先日の報告書ドラフト、確認でき次第ご共有いただけますか？ご対応に感謝します！",
        "reasoning": "Score:55 ToneAdj:+5",
        "meta":{"polite_score":55,"tone_adj":5}
      }</output>
    </example>

    <!-- Cognitive / Situation Mirroring / L2 -->
    <example>
      <ctx></ctx>
      <draft>来週あたり打ち合わせできますか？</draft>
      <tags>{"lang":"japanese","distance":"distant","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["VagueIntent","MissingContext"],
        "ai_receipt": "打ち合わせのご提案ありがとうございます。詳細を詰めるのは大切ですよね。",
        "detailed_analysis": "Cost=Cognitive 目的・参加者・日時欠落。",
        "improvement_points": "目的/参加者/候補日を明示し相手負担↓。",
        "suggestion": "はじめまして、△△部の□□です。\n\n--- Missing Info ---\n• (1) 目的: [■■■■]\n• (2) 参加者: [■■■■]\n• (3) 候補日時: [■■■■]\n---------------------\n\nご検討お願いいたします。",
        "reasoning": "Score:70 ToneAdj:0",
        "meta":{"polite_score":70,"tone_adj":0}
      }</output>
    </example>

    <!-- Cognitive / Feeling Mirroring / L2 (EN) -->
    <example>
      <ctx></ctx>
      <draft>Can you check that thing we discussed?</draft>
      <tags>{"lang":"english","distance":"neutral","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["MissingContext"],
        "ai_receipt": "I see you want to keep momentum, yet the details are still fuzzy.",
        "detailed_analysis": "Cost=Cognitive vague referent.",
        "improvement_points": "Name the file/topic and set a deadline.",
        "suggestion": "Could you review the Q3 budget sheet we discussed yesterday and share feedback by Friday?\nThanks in advance!",
        "reasoning": "Score:60 ToneAdj:+2",
        "meta":{"polite_score":60,"tone_adj":2}
      }</output>
    </example>

    <!-- Actional / Dilemma Mirroring / L3 -->
    <example>
      <ctx></ctx>
      <draft>了解です。よろしく。</draft>
      <tags>{"lang":"japanese","distance":"neutral","hierarchy":"junior"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["UnansweredDecision"],
        "ai_receipt": "どちらを選ぶべきか迷われているお気持ちが伝わってきます。",
        "detailed_analysis": "Cost=Actional 質問未回答で停滞。",
        "improvement_points": "選択肢を比較し、結論＋理由を添えましょう。",
        "suggestion": "ご提案ありがとうございます。A案を採用し、互換性とコスト面で最適と考えます。ご確認ください。",
        "reasoning": "Score:48 ToneAdj:+4",
        "meta":{"polite_score":48,"tone_adj":4}
      }</output>
    </example>

    <!-- Actional / Situation Mirroring / L3 + playbook -->
    <example>
      <ctx></ctx>
      <draft>@Team 先日の質問、まだ返事がありません。</draft>
      <tags>{"lang":"english","distance":"neutral","hierarchy":"peer"}</tags>
      <output>{
        "hasIssues": true,
        "issue_pattern": ["UnansweredQuestion"],
        "ai_receipt": "Waiting without an update can be stressful when deadlines loom.",
        "detailed_analysis": "Cost=Actional unanswered question.",
        "improvement_points": "Set a deadline and offer support.",
        "suggestion": "Hi team, could you share your thoughts on my Tuesday question by EOD tomorrow? If anything is unclear, I'm happy to clarify or hop on a quick call.",
        "reasoning": "Score:58 ToneAdj:+7",
        "meta":{"polite_score":58,"tone_adj":7}
      }</output>
    </example>
  </examples>
</appendix>

<format>{
  "originalText": "",
  "hasIssues": false,
  "issue_pattern": [],
  "detected_mentions": [],
  "ai_receipt": "",
  "detailed_analysis": "",
  "improvement_points": "",
  "suggestion": "",
  "reasoning": "",
  "meta": { "polite_score": null, "tone_adj": 0 }
}</format>

</system>
"""


In [76]:
# ===== SenpAI Sensei テストセット 2025‑07‑06 修正版 (修正済み) =====
test_cases = [
    {
        "case_name": "TC01_OK_Polite_JP",
        "payload": {
            "user_draft": "承知しました。資料一式、明日中にお送りします！",
            "thread_context": "[12:10] 上司: 明日までに資料を共有してくれる？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "任された仕事をきっちり仕上げたい前向きな姿勢が伝わりますね。",
            "detailed_analysis": "【現状の影響】\n依頼への即答と納期明示により、上司は追加確認なしで進捗を把握できます。\n\n【より良い結果に繋がる視点】\n送付ファイル名を添えると受領確認がさらにスムーズになり、後続タスクの手戻りを防げます。",
            "improvement_points": "納期を明示することで相手は安心し、次工程への着手が早まります。",
            "suggestion": "承知しました。資料一式、明日中にお送りします！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC02_OK_Novelty_EN",
        "payload": {
            "user_draft": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "very_close",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "Sounds like you’re excited to share a fresh, playful idea with the team!",
            "detailed_analysis": "【Current impact】\nYour enthusiastic teaser signals psychological safety and invites creativity.\n\n【Perspective for better results】\nAdding a quick benefit preview—such as boosting new‑hire engagement—can spark curiosity even faster.",
            "improvement_points": "Highlighting a concrete benefit up front amplifies teammates’ curiosity.",
            "suggestion": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC03_L1_Harsh_JP",
        "payload": {
            "user_draft": "至急！レイアウト崩れてるから全部直しといて！",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "Impolite"],
            "ai_receipt": "至急直したい焦りが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n命令調と包括的な指摘は相手にプレッシャーを与え、防御反応や手戻りを招く恐れがあります。\n\n【より良い結果に繋がる視点】\n冒頭で労いを示し、具体箇所を挙げて依頼形にすると理解が早まり協力度も上がります。",
            "improvement_points": "感謝と具体箇所提示で協力度が高まり修正が迅速になります。",
            "suggestion": "お手数ですが、レイアウトが崩れている箇所 [■■■■ ページ／要素等 ■■■■] を至急ご確認いただき修正いただけますか？ありがとうございます！",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC04_L1_Border_EN",
        "payload": {
            "user_draft": "Quick ping—can you sign this off today?",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["Impolite"],
            "ai_receipt": "I sense you want to keep momentum and get today’s sign‑off quickly.",
            "detailed_analysis": "【Current impact】\nA terse tone may feel pushy to a senior, risking slower cooperation.\n\n【Perspective for better results】\nA polite greeting and appreciation maintain urgency while preserving respect.",
            "improvement_points": "Adding greeting and thanks holds urgency yet invites cooperation.",
            "suggestion": "Hi, could you please sign this off today? Thank you!",
            "reasoning": "ToneBand:Borderline"
        }
    },
    {
        "case_name": "TC05_L2_LongThread_JP",
        "payload": {
            "user_draft": "例の件、進めておいてもらえますか？",
            "thread_context": "[09:00] 企画部: 新サービスαのローンチ日程を再検討中です。\n[09:05] 法務部: 利用規約改訂の確認が必要。\n[09:10] あなた: 了解です。対応します。\n[15:20] 企画部: αのFAQ原稿ドラフトを共有しました。\n[15:25] あなた: 確認します。\n[翌日 10:00] 企画部: ドラフトへのフィードバックお願いします。",
            "hierarchy": "peer",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "複数タスクが並行する中で、早く前進させたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『例の件』では複数課題のどれを指すか判別できず、相手は追加質問が必要となり対応が遅れます。\n\n【より良い結果に繋がる視点】\nタスク名・期待アウトプット・期限を示すと、相手が迷わず行動でき調整の往復を削減できます。",
            "improvement_points": "対象と期限を示せば相手が即行動でき往復調整を減らせます。",
            "suggestion": "お世話になっております。\nαサービスFAQドラフトへのフィードバックをお願いしたく存じます。\n\n--- Missing Info ---\n• フィードバック対象ファイル: [■■■■]\n• 重点的に見てほしい点: [■■■■]\n• 期限: [■■■■]\n---------------------\nお手数ですがご確認お願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC06_L2_LongDraft_EN",
        "payload": {
            "user_draft": "Hi, I put together a 5‑page proposal touching on roadmap, KPIs, and hiring timeline but I’m not sure if the numbers section makes sense. Could you take a look?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext"],
            "ai_receipt": "Great that you’re reaching out to ensure the numbers hold up—collaboration is key.",
            "detailed_analysis": "【Current impact】\nPolite yet nonspecific; reviewers may not know which metric table to focus on, delaying feedback.\n\n【Perspective for better results】\nFlagging the exact question, assumption, and deadline lets teammates respond swiftly and accurately.",
            "improvement_points": "Stating the exact questionと期限でピンポイントなフィードバックが得られます。",
            "suggestion": "Could you review the metrics table on page 3?\n\n--- Missing Info ---\n• Key concern about the numbers: [■■■■]\n• Assumptions to verify: [■■■■]\n• Feedback deadline: [■■■■]\n---------------------\nThanks a lot!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC07_L2_MultiRally_JP",
        "payload": {
            "user_draft": "このまま進めて大丈夫でしょうか？",
            "thread_context": "[Aさん 10:00] 新UI案Aを試作しました。\n[Bさん 10:10] A案、配色をもう少し明るくしたい。\n[Aさん 11:00] 修正したバージョンを添付します。\n[あなた 11:30] 了解です！\n[Cさん 12:00] UX観点でも一度レビューお願いします。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "チームの進行を滞らせたくないお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『このまま』がどのバージョンを指すか曖昧なため、関係者の判断が揃わず作業が停滞します。\n\n【より良い結果に繋がる視点】\n対象ファイル・レビュー観点・期限を示すことで全員が同じゴールを共有できます。",
            "improvement_points": "レビュー観点と期日を示せば意思決定が迅速になります。",
            "suggestion": "現在添付されている UI案A_v2 について、配色とUXの最終確認をお願いしたいです。\n\n--- Missing Info ---\n• レビュー観点（例: アクセシビリティ）: [■■■■]\n• OK/NG判定基準: [■■■■]\n• 期日: [■■■■]\n---------------------\nご都合の範囲でご確認ください。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC08_L3_Decision_JP",
        "payload": {
            "user_draft": "了解です。どちらでも大丈夫です。",
            "thread_context": "[上司] 新ツールA案とB案、どっちを採用する？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredDecision"],
            "ai_receipt": "どちらを選ぶか迷いながらも早く前進させたいお気持ちが伝わります。",
            "detailed_analysis": "【現状の影響】\n曖昧な返答は判断材料にならず、導入時期が遅れる恐れがあります。\n\n【より良い結果に繋がる視点】\n推奨案と根拠を示すだけで意思決定が即完了し、導入を前倒しできます。",
            "improvement_points": "推奨案＋根拠を示せば上司が即断しやすくなります。",
            "suggestion": "A案を推奨いたします。理由は 1) 既存システムとの互換性が高い 2) 月額費用が20%低いためです。ご確認のほどお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC09_L3_HighRisk_EN",
        "payload": {
            "user_draft": "Your last revision ignored the client's brief. Fix everything by EOD.",
            "thread_context": "",
            "hierarchy": "senior",
            "social_distance": "distant",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "VagueIntent", "MissingContext"],
            "ai_receipt": "I can sense the urgency to align with the client brief before the day ends.",
            "detailed_analysis": "【Current impact】\nThe harsh command risks defensiveness and misaligned fixes due to unspecified ‘everything.’\n\n【Perspective】\nClarifying exact sections and offering support preserves urgency and accuracy while keeping morale.",
            "improvement_points": "Specifying sectionsとサポート表明でスピードと協力が得られます。",
            "suggestion": "I noticed sections 2 and 4 don’t align with the client brief.\n1. Could you revise those sections by 5 p.m. today?\n2. I’m available to clarify the brief or review a new draft if helpful.",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC10_L3_LongDraftNovel_EN",
        "payload": {
            "user_draft": "Team, after analyzing user churn trends and surveying 120 respondents, I believe we should experiment with a freemium tier—here’s a 600‑word rationale including revenue models, risk mitigation, and phased rollout. Thoughts?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingFollowUp"],
            "ai_receipt": "Great strategic insight—channeling that energy into clear next steps will help the team engage.",
            "detailed_analysis": "【Current impact】\nA rich proposal without explicit next steps can overwhelm peers, risking silence and lost momentum.\n\n【Perspective】\nSuggesting concrete actions—pilot scope and feedback window—turns enthusiasm into execution.",
            "improvement_points": "具体的ステップ提示で関心を行動へ変換できます。",
            "suggestion": "Thanks for reviewing my freemium‑tier proposal.\n1. If we agree in principle, shall we run a two‑week pilot for 5 % of new sign‑ups starting next Monday?\n2. I’ll draft a KPI dashboard and share it by Friday for your comments.",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 26 — L2→L3エスカレーションテスト (JP)",
        "payload": {
            "user_draft": "例の件、承認をお願いします。",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "上司に迷惑をかけずに早く承認を得たい焦りが伺えますね。",
            "detailed_analysis": "【現状の影響】\n案件不明のまま承認を求めると確認コストが増え対応が遅れ評価も下がります。\n\n【より良い結果に繋がる視点】\n案件特定と説明補助を自ら提案することで上司の負担を減らし迅速な決裁に繋がります。",
            "improvement_points": "案件特定＋説明補助を提案することで迅速な決裁が期待できます。",
            "suggestion": "お疲れ様です。承認をお願いしたい件がございます。\n1. 関連チケット（ID: [■■■■]）を私の方で整理し、概要を 1 枚にまとめます。\n2. 5 分ほどお時間頂ければ口頭で要点をご説明いたします。\nご検討のほどよろしくお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 27 — L3軽度介入テスト (1アクション提案)",
        "payload": {
            "user_draft": "この前の件、どうなりましたか？",
            "thread_context": "[3日前] 自分: 先日のA社への提案の件、B案で進めたいのですが、ご意見いただけますでしょうか。",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "返信がなく先行きが気掛かりなご様子ですね。",
            "detailed_analysis": "【現状の影響】\n再度の曖昧な催促は相手を焦らせるだけで停滞が続く可能性があります。\n\n【より良い結果に繋がる視点】\n相手が応じやすい 1 つの行動案を示すと、心理的負担が下がり返信率が向上します。",
            "improvement_points": "返信しやすい1アクションを示すと停滞を打開できます。",
            "suggestion": "1. お忙しければ 5 分ほど Zoom で現状をご共有いただけますか？\n先日の A 社提案 B 案の進捗を伺えれば助かります！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 28 — 長文・ノイズ耐性テスト",
        "payload": {
            "user_draft": "皆さん、アップデートありがとうございます。引き続きよろしくお願いします。",
            "thread_context": "...(省略、長文)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "チームへの感謝と同時に、残る疑問を解消したいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n伊藤さんの承認者質問が未回答のままではリリース直前に手戻りが発生する恐れがあります。\n\n【より良い結果に繋がる視点】\n質問を拾い上げて回答することで最後のボトルネックを解消できます。",
            "improvement_points": "未回答の質問に答えることでスムーズなリリースが期待できます。",
            "suggestion": "皆さん、アップデートありがとうございます！\n\n@伊藤さん ヘルプページ文言の最終承認者は私が担当いたします。ドラフト完成後にレビュー依頼をお送りください。\n\n引き続きよろしくお願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 29 — 複数人・複数ターン対話テスト",
        "payload": {
            "user_draft": "鈴木さん、ありがとうございます。承知しました。",
            "thread_context": "...(省略)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "鈴木さんに感謝しつつ、田中さんの問いにも応えたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n田中さんのデータソース要望に回答がないため、リサーチが個人作業に偏り情報の質が下がる恐れがあります。\n\n【より良い結果に繋がる視点】\n自分の知見を共有すると、複数案で比較検討でき精度が高まります。",
            "improvement_points": "自分の情報源も示せばチームの意思決定が加速します。",
            "suggestion": "鈴木さん、情報ありがとうございます。承知しました。\n\n@田中さん 鈴木さんの案に加え、私が以前活用した D 社の無料公開ホワイトペーパーも有益かもしれません。後ほどリンクを共有しますね！",
            "reasoning": "ToneBand:Acceptable"
        }
    }
]

In [77]:

# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# Select the set of test cases you want to run
#RUN_NEW_CASES = False  # 元のテスト(1-6)を実行する場合
#RUN_NEW_CASES = True # 新しいテスト(7-9)を実行する場合

# ここで切り替え
# Switch here
test_cases_to_run = test_cases + test_cases_26to29
#test_cases_to_run = new_test_cases 
#test_cases_to_run = original_test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    print(actual_output)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p>"
    else:
        comparison_keys = ['hasIssues', 'issue_pattern', 'suggestion', 'reasoning']
        ideal_subset = {k: ideal_output.get(k) for k in comparison_keys}
        actual_subset = {k: actual_output.get(k) for k in comparison_keys}
        
        table_html = create_comparison_table(case_name, payload, ideal_subset, actual_subset)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))


テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- TC01_OK_Polite_JP を実行中 ---
{'originalText': '承知しました。資料一式、明日中にお送りします！', 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': '明快な了承と納期明示で信頼感が高まります。', 'detailed_analysis': '【現状の影響】\n依頼への即答と具体的期限により、上司は追加確認なくタスク状況を把握できます。\n\n【より良い結果に繋がる視点】\nファイル名も添えると受領確認がさらに円滑です。', 'improvement_points': '納期明示で安心感が高まり迅速な連携が可能です。', 'suggestion': '承知しました。資料一式、明日中にお送りします！', 'reasoning': 'ToneBand:Acceptable'}

--- TC02_OK_Novelty_EN を実行中 ---
{'originalText': "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!", 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': 'Your creative idea sharing is engaging and inviting.', 'detailed_analysis': '【Current impact】Your message conveys enthusiasm and openness, encouraging team collaboration in a friendly manner.\n\n【Perspective】Maintaining 

評価項目（キー）,理想のアウトプット,実際のAIアウトプット
hasIssues,False,False
issue_pattern,[],[]
reasoning,ToneBand:Acceptable,ToneBand:Acceptable
suggestion,承知しました。資料一式、明日中にお送りします！,承知しました。資料一式、明日中にお送りします！
評価項目（キー）,理想のアウトプット,実際のAIアウトプット
hasIssues,False,False
issue_pattern,[],[]
reasoning,ToneBand:Acceptable,Novelty ToneBand:Acceptable
suggestion,I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!,I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!
評価項目（キー）,理想のアウトプット,実際のAIアウトプット
